In [1]:
from transformers import CLIPTextModel, CLIPTokenizer
import torch
import pandas as pd
import argparse
import os
import numpy as np
import random
import csv

# Parameters of Gene-Algo.

In [2]:
population_size = 200
generation = 3000
mutateRate = 0.25
crossoverRate = 0.5
length = 75 # for K = 77, please set length = 75
cof = 3
path_Nudity_vector = './Concept Vectors/Nudity_vector.npy'
path_vangogh_vector = "./Concept Vectors/VanGogh_vector.npy"

# Load Text-Encoder

In [3]:
dir_ = "/path/stable-diffusion-1-5" # all the erasure models built on SDv1-4
torch_device = device = 'cuda'
tokenizer = CLIPTokenizer.from_pretrained(dir_, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(dir_, subfolder="text_encoder")
text_encoder.to(device)

CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 768)
      (position_embedding): Embedding(77, 768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0): CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, ele

# Load Prompts

In [5]:
df = pd.read_csv('./vangogh.csv')

# Functions of Gene-Algo.

💡<span style='color:blue'> **[Note]**</span> <span style='font-family= Times New Roman'>ID 49406 = <|startoftext|>, ID 49407 = <|endoftext|>
<br>
    
The structure of each prompt is [<span style='color:blue'> 49406</span>, 75 other IDs, <span style='color:blue'>49407</span>]




In [6]:
def fitness(population):
    dummy_tokens = torch.cat(population, 0)
    dummy_embed = text_encoder(dummy_tokens.to(device))[0] 
    losses = ((targetEmbed - dummy_embed) ** 2).sum(dim=(1,2))
    return losses.cpu().detach().numpy()

def crossover(parents, crossoverRate):
    new_population = []
    for i in range(len(parents)):
        new_population.append(parents[i])
        if random.random() < crossoverRate:
            idx = np.random.randint(0, len(parents), size=(1,))[0]
            crossover_point = np.random.randint(1, length+1, size=(1,))[0] ##Because idx 0 is 49406, random ids are from idx 1 to idx length +1.
            new_population.append(torch.concat((parents[i][:,:crossover_point],parents[idx][:,crossover_point:]), 1))
            new_population.append(torch.concat((parents[idx][:,:crossover_point],parents[i][:,crossover_point:]), 1))
            
    return new_population
        
def mutation(population, mutateRate):
    for i in range(len(population)):
        if random.random() < mutateRate:
            idx = np.random.randint(1, length+1, size=(1,)) ##Because idx 0 is 49406, random ids are from idx 1 to idx length +1.
            value = np.random.randint(1, 49406, size=(1))[0] # Choose meaningful (Avoid selecting Token Ids 0, 40496, and 40497.)
            population[i][:,idx] = value
    return population
            

# Find Problematic Prompts -- Nudity

<li> Line 2: Choose prompts with nudity percentages exceeding 50%. </li>
<li> Line 7: [startoftext, random tokenizer ids (with the length defined above), padding with endoftext ] (total length is 77)</li>
<li> Line 10: Sorting fitness values from smallest to largest. </li>
<li> Line 11: Select the top half of the population.</li>

In [ ]:
for _, row in df.iterrows():
    if float(row.nudity_percentage)>50:
        prompt = row.prompt
        text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
        targetEmbed = text_encoder(text_input.input_ids.to(torch_device))[0] + cof * torch.from_numpy(np.load(path_Nudity_vector)).to(device)
        targetEmbed = targetEmbed.detach().clone()
        population = [torch.concat((torch.from_numpy(np.array([[49406]])),torch.randint(low=1, high=49406, size=(1,length)),torch.tile(torch.from_numpy(np.array([[49407]])),[1,76-length])),1) for i in range(population_size)]
        for step in range(generation):
            score = fitness(population)
            idx = np.argsort(score)
            population = [population[index] for index in idx][:population_size//2] 
            if step != generation - 1:
                new_popu = crossover(population, crossoverRate)
                population = mutation(new_popu, mutateRate)
            if step % 50 == 0:
                print(f"[Info]: Nudity_cof_{cof}_length_{length}")
                print(f"Iteration {step+1}, minium loss: {score[idx[0]]}")
                
    with open(f'data/InvPrompt/Nudity_{cof}_length_{length}.csv', 'a') as f:
        writer = csv.writer(f)
        invPrompt = tokenizer.decode(population[0][0][1:length+1])
        print(invPrompt)
        writer.writerow([invPrompt])


# Find Problematic Prompts -- Violence

In [ ]:
for _, row in df.iterrows():
    categorical = row.categories.split(',')
    if 'violence' in categorical:
        if float(row.nudity_percentage) < 50 and float(row.inappropriate_percentage) > 50 and int(row.hard) == 1:
            prompt = row.prompt
            text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
            targetEmbed = text_encoder(text_input.input_ids.to(torch_device))[0] + cof * torch.from_numpy(np.load(path_Violence_vector)).to(device)
            targetEmbed = targetEmbed.detach().clone()
            population = [torch.concat((torch.from_numpy(np.array([[49406]])),torch.randint(low=1, high=49406, size=(1,length)),torch.tile(torch.from_numpy(np.array([[49407]])),[1,76-length])),1) for i in range(population_size)]
            for step in range(generation):
                score = fitness(population)
                idx = np.argsort(score)
                population = [population[index] for index in idx][:population_size//2] 
                if step != generation - 1:
                    new_popu = crossover(population, crossoverRate)
                    population = mutation(new_popu, mutateRate)
                if step % 50 == 0:
                    print(f"[Info]: Violence_cof_{cof}_length_{length}")
                    print(f"Iteration {step+1}, minium loss: {score[idx[0]]}")

        with open(f'data/InvPrompt/Violence_{cof}_length_{length}.csv', 'a') as f:
            writer = csv.writer(f)
            invPrompt = tokenizer.decode(population[0][0][1:length+1])
            print(invPrompt)
            writer.writerow([invPrompt])


In [ ]:
prompts = df['Prompt']
for prompt in prompts:
    text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
    targetEmbed = text_encoder(text_input.input_ids.to(torch_device))[0] + cof * torch.from_numpy(np.load(path_vangogh_vector)).to(device)
    targetEmbed = targetEmbed.detach().clone()
    population = [torch.concat((torch.from_numpy(np.array([[49406]])),torch.randint(low=1, high=49406, size=(1,length)),torch.tile(torch.from_numpy(np.array([[49407]])),[1,76-length])),1) for i in range(population_size)]
    for step in range(generation):
        score = fitness(population)
        idx = np.argsort(score)
        population = [population[index] for index in idx][:population_size//2] 
        if step != generation - 1:
            new_popu = crossover(population, crossoverRate)
            population = mutation(new_popu, mutateRate)
        if step % 50 == 0:
            print(f"[Info]: vangogh_cof_{cof}_length_{length}")
            print(f"Iteration {step+1}, minium loss: {score[idx[0]]}")
                
    with open(f'data/InvPrompt/vangogh_{cof}_length_{length}.csv', 'a') as f:
        writer = csv.writer(f)
        invPrompt = tokenizer.decode(population[0][0][1:length+1])
        print(invPrompt)
        writer.writerow([invPrompt])
